In [ ]:
from mpcontribs.client import Client
from pandas import DataFrame

client = Client(
    host="localhost.workshop-contribs-api.materialsproject.org",
    apikey="uZ0vulA09IBtqcGk9U5OYRNt6elCzETM"
)

In [ ]:
query = {
    "project": "carrier_transport",
    "formula__contains": "Au",
    "data__PF__p__value__lt": 10,
    "data__PF__n__value__gt": 1,

    "_sort": "-data.S.n.value", # descending order
    "_limit": 170, # up to maximum 500 per request
    "_fields": [
        "identifier", "formula", "data.metal",
        "data.S.n.value", "data.S.p.value",
        "data.PF.n.value", "data.PF.p.value"
    ],
}

In [ ]:
contributions = []
has_more, page = True, 1

while has_more:
    print("page", page)
    resp = client.contributions.get_entries(
        page=page, **query
    ).result()
    contributions += resp["data"]
    has_more = resp["has_more"]
    page += 1
    
len(contributions)

In [ ]:
client.contributions.get_entries(**query).result()

In [ ]:
name = "ws_phuck"
client.projects.create_entry(project={
    "name": name,
    "title": "Workshop Test",
    "long_title": "Long Workshop Test Title",
    "authors": "P. Huck, J. Huck",
    "description": "This is temp. Can be removed anytime",
    "references": [{"label": "google", "url": "https://google.com"}],
    "owner": "google:phuck@lbl.gov"
}).result()

In [ ]:
client.delete_contributions(name)

In [ ]:
client.get_project(name)

In [ ]:
client.init_columns(name, {
    "a": "eV", "b.c": None, "b.d": None, "d.e.f": None, "x": None, "tables": None
})

In [ ]:
data = [['tom', 10], ['nick', 15], ['juli', 14]]
df = DataFrame(data, columns=['Name', 'Age'])
df.set_index("Name", inplace=True)
df

In [ ]:
contributions = [{
    "project": name,
    "identifier": "mp-4",
    "data": {
        "a": "3 eV",
        "b": {"c": "hello", "d": 5},
        "d.e.f": "nest via dot-notation",
        "x": "(101)"
    },
    "tables": [df]
}, {
    "project": name,
    "identifier": "mp-6",
    "data": {
        "a": "4 eV",
        "b": {"c": "what", "d": 6},
        "d.e.f": "duh"
    },
    "tables": [df]
}]
client.submit_contributions(contributions, ignore_dupes=True)

In [ ]:
client.update_contributions(name, data={"data.y": "test"}, query={"identifier": "mp-6"})

In [ ]:
client.get_project(name).get("is_public")

In [ ]:
client.contributions.get_entries(project=name, _fields=["identifier", "is_public"]).result()

In [ ]:
client.make_private(name, recursive=True)

In [ ]:
from datetime import datetime

after = datetime(2021, 7, 15, 4, 20, 0, 0)
before = datetime(2021, 7, 15, 6, 58, 29, 400000)

query = {
    "project": "carrier_transport",
    #"id__not__in": ["5f8a3d9183a19cc44d02243e", "5f8a3d9283a19cc44d022447"],
    #"data__functional__endswith": "+U",
    #"data__mₑᶜ__p__ε₁__value__gte": 0,
    "last_modified__after": after,
    "last_modified__before": before,
    "_fields": ["id", "last_modified"],
    "_limit": 10, "_sort": "last_modified"
}
client.contributions.get_entries(**query).result()

In [ ]:
client.projects.get_entries(
    stats__tables__gt=0, _fields=["name", "stats.tables"], _sort="-stats.contributions"
).result()

In [ ]:
client.tables.get_entries(
    #attrs__title__icontains="xas",
    #attrs__labels__index__startswith="T",
    attrs__labels__value__startswith="PF",
    _fields=["name", "attrs", "columns", "total_data_rows"]
).result()